In [ ]:
pip install spacy

In [ ]:
pip install torch==2.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 127.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install torchtext==0.15.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 73.5 MB/s eta 0:00:00


In [ ]:
pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
import math
import time
import spacy
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
from torchtext.data.functional import to_map_style_dataset
from torch.nn.functional import pad
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
from torchtext.data.utils import get_tokenizer
from torch.utils.data import DataLoader,Dataset,random_split



In [ ]:
from google.colab import drive
import json

drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import torch
import torch.nn.functional as F
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset, random_split
from indicnlp.tokenize import indic_tokenize  # Telugu tokenizer from indic-nlp-library
import random
random.seed(42)
torch.manual_seed(42)

# Function to read your custom dataset
def read_telugu_english_data(file_paths):
    with open(file_paths, 'r', encoding='utf-8') as file:
        raw_datas = []
        for line in file:
            telugu_sentence, english_sentence = line.strip().split('>>')
            raw_datas.append((telugu_sentence, english_sentence))
    return raw_datas

# Splitting the dataset
def split_datasets(data, train_split=0.7, val_split=0.15, test_split=0.15):
    total_sizes = len(data)
    train_sizes = int(total_sizes * train_split)
    val_sizes = int(total_sizes * val_split)
    test_sizes = total_sizes - train_sizes - val_sizes
    train_data, remaining_data = random_split(data, [train_sizes, total_sizes - train_sizes])
    val_data, test_data = random_split(remaining_data, [val_sizes, test_sizes])
    return list(train_data), list(val_data), list(test_data)

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def get_raw_textss(self):
        return [(srcs, trgs) for srcs, trgs in self.data]

tokenizer_te = spacy.blank("xx").tokenizer  # Telugu tokenizer
tokenizer_ens = spacy.blank("en").tokenizer  # English tokenizer

# Build vocabulary function
def build_vocabularys(tokenizer, dataset, min_freq=2):
    def yield_tokenss(data):
        for srcs, trgs in data:
            yield [token.text for token in tokenizer(srcs)]
            yield [token.text for token in tokenizer(trgs)]

    vocabs = build_vocab_from_iterator(yield_tokenss(dataset.get_raw_textss()), specials=["<unk>", "<pad>", "<bos>", "<eos>"], min_freq=min_freq)
    vocabs.set_default_index(vocabs['<unk>'])  # Set default index for unknown tokenss
    return vocabs

# Read the dataset
file_paths = '/content/drive/MyDrive/eng-tel.txt'
raw_datas = read_telugu_english_data(file_paths)
train_data_raw, val_data_raw, test_data_raw = split_datasets(raw_datas)

# Create datasets
train_datasets = CustomDataset(train_data_raw)
valid_datasets = CustomDataset(val_data_raw)
test_datasets = CustomDataset(test_data_raw)

# Load vocabularies
vocab_src = build_vocabularys(tokenizer_ens, train_datasets)
vocab_trg = build_vocabularys(tokenizer_te, train_datasets)
# Build vocabs
vocab_en = build_vocabularys(tokenizer_ens, train_datasets)
vocab_te = build_vocabularys(tokenizer_te, train_datasets)
import pickle

# Save the vocabs to your Google Drive
with open('/content/drive/MyDrive/vocabs-eng.pkl', 'wb') as f:
    pickle.dump(vocab_en, f)
with open('/content/drive/MyDrive/vocabs-tel.pkl', 'wb') as f:
    pickle.dump(vocab_te, f)
# Batch generation function
def generate_batchs(data_batch):
    de_batch, en_batch = [], []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for (de_item, en_item) in data_batch:
        # Convert list of indices into tensors
        de_indices = torch.tensor([vocab_src[token.text] for token in tokenizer_te(de_item)], dtype=torch.long)
        en_indices = torch.tensor([vocab_trg[token.text] for token in tokenizer_ens(en_item)], dtype=torch.long)

        # Concatenate BOS, indices, EOS
        de_temp = torch.cat([torch.tensor([vocab_src['<bos>']], dtype=torch.long), de_indices, torch.tensor([vocab_src['<eos>']], dtype=torch.long)], dim=0).to(device)
        en_temp = torch.cat([torch.tensor([vocab_trg['<bos>']], dtype=torch.long), en_indices, torch.tensor([vocab_trg['<eos>']], dtype=torch.long)], dim=0).to(device)

        # Pad sequences to ensure consistent length
        padded_de = F.pad(de_temp, (0, MAX_PADDING - len(de_temp)), value=vocab_src['<pad>'])
        padded_en = F.pad(en_temp, (0, MAX_PADDING - len(en_temp)), value=vocab_trg['<pad>'])

        de_batch.append(padded_de)
        en_batch.append(padded_en)

    return torch.stack(de_batch), torch.stack(en_batch)

# DataLoader setup
BATCH_SIZE = 32
train_iters = DataLoader(train_datasets, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, collate_fn=generate_batchs)
valid_iters = DataLoader(valid_datasets, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, collate_fn=generate_batchs)
test_iters = DataLoader(test_datasets, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, collate_fn=generate_batchs)

BOS_IDX = vocab_trg['<bos>']
EOS_IDX = vocab_trg['<eos>']
PAD_IDX = vocab_trg['<pad>']
MAX_PADDING = 150
BATCH_SIZE= 32

In [ ]:
class Embeddingss(nn.Module):
  def __init__(self, vocab_size: int, d_model: int):
    """
    Args:
      vocab_size:    size of vocabulary
      d_model:       dimension of embeddings
    """
    super().__init__()
    self.lut = nn.Embedding(vocab_size, d_model)
    self.d_model = d_model

  def forward(self, xs):
    """
    Args:
      xs:        input tensor (batch_sizes, sseq_lenght)

      returns:  embedding vector
    """
    # xs = xs.to(self.lut.weight.devices)
    return (self.lut(xs) * math.sqrt(self.d_model))

In [ ]:
class PositionalEncodings(nn.Module):
  def __init__(self, d_model: int, dropout: float = 0.1, max_length: int = 5000):
    """
    Args:
      d_model:     dimension of embeddings
      dropout:     randomly zeroes-out some of the input
      max_lenght:  amx sequence length
    """

    super().__init__()
    self.dropout=nn.Dropout(p=dropout)
    pes=torch.zeros(max_length,d_model)
    for k in np.arange(max_length):
      for i in np.arange(d_model//2):
        thetas = k / (100** ((2*i)/d_model))


        pes[k, 2*i] = math.sin(thetas)


        pes[k, 2*i+1] = math.cos(thetas)
        self.register_buffer("pes",pes)

  def forward(self, xs):
    """
    Args:
      xs:        embeddings (batch_sizes, seq_lenght, d_model)
      returns:  embeddings + positonal encodings (batch_sizes, seq_lengths, d_model)
    """
    self.pes=self.pes.to(xs.device)
    xs = xs + self.pes[:xs.size(1), :].requires_grad_(False)
    return self.dropout(xs)

In [ ]:
class MultiHeadAttentions(nn.Module):
  def __init__(self, d_model, n_heads, dropout: float = 0.1):
    """
    Args:
      d_model:      dimension of embeddings
      n_heads:      number of self attention heads
      dropout:      probability of dropout occuring
    """
    super().__init__()
    self.d_model = d_model
    self.n_heads = n_heads
    self.d_key = d_model // n_heads

    # create query, key, value, outputs weights
    self.Wq = nn.Linear(d_model, d_model)
    self.Wk = nn.Linear(d_model,d_model)
    self.Wv = nn.Linear(d_model,d_model)
    self.Wo = nn.Linear(d_model,d_model)

    self.dropout = nn.Dropout(p = dropout)

  def forward(self, query, key, value, masks = None):
    """
    Args:
      query:    query vector (batch_sizes, q_length, d_model)
      key:      key vector (batch_sizes, k_length, d_model)
      value:    value vector (batch_sizes, s_length, d_model)
      masks:     masks for decoders

    Returns:
      outputs:    attention values (batch_sizes, q_lenght, d_model)
      attn_probss:  softmax scoress (batchsize, n_heads, q_length, k_length)
    """
    batch_sizes = query.shape[0]

    # calculate query, key, and value tensors
    Qs = self.Wq(query)
    Ks = self.Wk(key)          # (32, 10, 512) xs (512, 512) = (32, 10, 512)
    Vs = self.Wv(value)

    # split each tensor into n_heads to compute attention

    # query tensor
    Qs = Qs.view(batch_sizes,
               -1,                                    # (32, 10, 512) -> (32, 10, 8 ,64)
               self.n_heads,                          # -1 = q_lenght
               self.d_key).permute(0, 2, 1, 3)        # (32, 10, 8, 64) -> (32, 8, 10, 64)

    # key tensor
    Ks = Ks.view(batch_sizes,
               -1,
               self.n_heads,
               self.d_key).permute(0, 2, 1, 3)

    # value tensor
    Vs = Vs.view(batch_sizes,
               -1,
               self.n_heads,
               self.d_key).permute(0, 2, 1, 3)

    # computes attention
    # scalled dot product -> QK^{T}
    scaled_dot_prod = torch.matmul(Qs, Ks.permute(0, 1, 3, 2)) / math.sqrt(self.d_key)

    # scaled_dot_prod = scaled_dot_prod.to(masks.devices)

    # fill thoes positions of product as (-1e10) where masks positions are 0
    if masks is not None:
      scaled_dot_prod = scaled_dot_prod.masked_fill(masks == 0, -1e10)

    attn_probss = torch.softmax(scaled_dot_prod, dim = -1)

    # attn_probss = attn_probss.to(Vs.devices)

    # multiply by values to get attention
    As = torch.matmul(self.dropout(attn_probss), Vs)


    # reshape attention back to (32, 10, 512)
    As = As.permute(0,2,1,3).contiguous()               # (32, 8, 10, 64) -> (32, 10, 8 ,64)
    As = As.view(batch_sizes, -1, self.n_heads*self.d_key)     # (32, 10, 8, 64) -> (32, 10, 8*64) = (32, 10, 512)

    outputs = self.Wo(As)

    return outputs, attn_probss

In [ ]:
class PositionwiseFeedForwards(nn.Module):
  def __init__(self, d_model: int, d_ffn: int, dropout: float = 0.1):
    """
    Args:
      d_model:      dimension of embeddings
      d_ffn:        dimension of feed-forwards network
      dropout:      probability of dropout occuring
    """

    super().__init__()
    self.linear_layer_1 = nn.Linear(d_model, d_ffn)
    self.linear_layer_2 = nn.Linear(d_ffn, d_model)
    self.dropout = nn.Dropout(p=dropout)

  def forward(self, xs):
    """
    Args:
      xs:        outputs from attention (batch_sizes, seq_lengths, d_model)

    Returns:
      expanded-and-contracted representation (batch_sizes, seq_lengths, d_model)
    """

    return self.linear_layer_2(self.dropout(self.linear_layer_1(xs).relu()))

In [ ]:
class EncoderLayers(nn.Module):
  def __init__(self, d_model: int, n_heads: int, d_ffn: int, dropout: float):
    """
    Args:
      d_model:      dimension of embeddings
      n_heads:      number of heads
      d_ffn:        dimension of feed-forwards network
      dropout:      probability of dropout ocurring
    """
    super().__init__()
    self.attention = MultiHeadAttentions(d_model, n_heads, dropout)
    self.attn_layer_norm = nn.LayerNorm(d_model)
    self.positionwise_fnn = PositionwiseFeedForwards(d_model, d_ffn, dropout)
    self.fnn_layer_norm = nn.LayerNorm(d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self, srcs, src_masks):
    """
    Args:
      srcs:      positionally embedded sequences (batch_sizes, seq_lengths, d_model)
      src_masks: masks for the sequences (batch_sizes, 1, 1, seq_lenght)
    Returns:
      srcs:      Sequences after self-attention (batch_sizes, seq_lengths, d_model)
    """

    _srcs, attn_probss = self.attention(srcs, srcs, srcs, src_masks)

    srcs = self.attn_layer_norm(srcs + self.dropout(_srcs))

    _srcs = self.positionwise_fnn(srcs)

    srcs = self.fnn_layer_norm(srcs + self.dropout(_srcs))

    return srcs, attn_probss


class Encoders(nn.Module):
  def __init__(self, d_model: int, n_layers: int, n_heads: int, d_ffn: int, dropout: float = 0.1):
    """
    Args:
      d_model:      dimension of embeddings
      n_layers:     number of encoders layers
      n_heads:      number of heads
      d_ffn:        dimension of feed-forwards network
      dropout:      probability of dropout occuring
    """
    super().__init__()

    # create n_layers encoders
    self.layers = nn.ModuleList([EncoderLayers(d_model, n_heads, d_ffn, dropout) for _ in range(n_layers)])

    self.dropout = nn.Dropout(dropout)

  def forward(self, srcs, src_masks):
    """
    Args:
      srcs:      positionally embedded sequences (batch_sizes, seq_lengths, d_model)
      src_masks: masks for the sequences (batch_sizes, 1, 1, seq_lenght)
    Returns:
      srcs:      Sequences after self-attention (batch_sizes, seq_lengths, d_model)
    """

    # Pass the sequence through each encoders
    for layer in self.layers:
      srcs, attn_probss = layer(srcs, src_masks)

    self.attn_probss = attn_probss
    return srcs

In [ ]:
class DecoderLayers(nn.Module):
  def __init__(self, d_model: int, n_heads: int, d_ffn: int, dropout: float):
    """
    Args:
      d_model:      dimension of embeddings
      n_heads:      number of heads
      d_ffn:        dimension of feed-forwards network
      dropout:      probability of dropout occuring
    """
    super().__init__()
    self.masked_attention = MultiHeadAttentions(d_model, n_heads, dropout)
    self.masked_attn_layer_norm = nn.LayerNorm(d_model)
    self.attention = MultiHeadAttentions(d_model, n_heads, dropout)
    self.attn_layer_norm = nn.LayerNorm(d_model)
    self.positionwise_fnn = PositionwiseFeedForwards(d_model, d_ffn, dropout)
    self.fnn_layer_norm = nn.LayerNorm(d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self, trgs, srcs, trg_masks, src_masks):
    """
    Args:
      trgs:          embedded sequences (batch_sizes, trg_seq_length, d_model)
      srcs:          embedded sequences (batch_sizes, src_seq_length, d_model)
      trg_masks:     masks for the sequences (batch_sizes, 1, trg_seq_length, trg_seq_lengt
      src_masks:     masks for the sequences (batch_sizes, 1, 1, src_seq_length)

    Returns:
      trgs: sequences after self-attention (batch_sizes, trg_seq_length, d_model)
      attn_probss: self-attention softmax scoress (batch_sizes, n_heads, trg_seq_length, src_seq_lenght)
    """

    _trgs, attn_probss = self.masked_attention(trgs, trgs, trgs, trg_masks)

    trgs = self.masked_attn_layer_norm(trgs + self.dropout(_trgs))

    _trgs, attn_probss = self.attention(trgs, srcs, srcs, src_masks)

    trgs = self.attn_layer_norm(trgs + self.dropout(_trgs))

    _trgs = self.positionwise_fnn(trgs)

    trgs = self.fnn_layer_norm(trgs + self.dropout(_trgs))

    return trgs, attn_probss

class Decoders(nn.Module):
  def __init__(self, vocab_size: int, d_model: int, n_layers: int, n_heads: int, d_ffn: int, dropout: float = 0.1):
    """
    Args:
      vocab_size:     size of the target vocabulary
      d_model:        dimension of embeddings
      n_layers:       number of encoders layers
      n_heads:        number of heads
      d_ffn:          dimension of feed-forwards network
      dropout:        probability of dropout occurring
    """
    super().__init__()

    # create n_layers encoders
    self.layers = nn.ModuleList([DecoderLayers(d_model, n_heads, d_ffn, dropout) for _ in range(n_layers)])

    self.dropout = nn.Dropout(dropout)

    # set outputs layer
    self.Wo = nn.Linear(d_model, vocab_size)

  def forward(self, trgs, srcs, trg_masks, src_masks):
      """
      Args:
        trgs:          embedded sequences (batch_sizes, trg_seq_length, d_model)
        srcs:          embedded sequences (batch_sizes, src_seq_length, d_model)
        trg_masks:     masks for the sequences (batch_sizes, 1, trg_seq_length, trg_seq_lengt
        src_masks:     masks for the sequences (batch_sizes, 1, 1, src_seq_length)

      Returns:
        outputs:       sequences after decoders (batch_sizes, trg_seq_length, vocab_size)
        attn_probss:   self-attention softmax scoress (batch_sizes, n_heads, trg_seq_length, src_seq

      """

      # pass the sequences through each decoders
      for layer in self.layers:
        trgs , attn_probss = layer(trgs, srcs, trg_masks, src_masks)

      self.attn_probss = attn_probss
      return self.Wo(trgs)


In [ ]:
class Transformers(nn.Module):
  def __init__(self, encoders: Encoders, decoders: Decoders, src_embeds: Embeddingss,
               trg_embeds: Embeddingss, src_pad_idx: int, trg_pad_idx: int, device):
    """
    Args:
      encoders:        encoders stack
      decoders:        decoders stack
      src_embeds:      source embeddings and encodings
      trg_embeds:      target embeddings and encodings
      src_pad_idx:    padding index

      trg_pad_idx:    padding index
      devices:         cpu or gpu

    Returns:
      outputs:         sequences after decoders (batch_sizes, trg_seq_length, vocab_size)
    """
    super().__init__()

    self.encoders = encoders
    self.decoders = decoders
    self.src_embeds = src_embeds
    self.trg_embeds = trg_embeds
    self.device = device
    self.src_pad_idx = src_pad_idx
    self.trg_pad_idx = trg_pad_idx

  def make_src_masks(self, srcs):
    """
    Args:
      srcs:        raws sequence with padding     (batch_sizes, seq_lengths)

    Returns:
      src_masks:   masks for each sequence        (batch_sizes, 1, 1, seq_lenght)
    """
    # assign 1 to tokenss that need attended to and 0 to padding tokenss, then add 2 dimensions
    src_masks = (srcs != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
    return src_masks

  def make_trg_masks(self, trgs):
    """
    Args:
      trgs:        raws sequence with padding     (batch_sizes, seq_lengths)

    Returns:
      trg_masks:   masks for each sequence        (batch_sizes, 1, seq_lengths, seq_lenght)
    """
    seq_lengths = trgs.shape[1]

    # assign True to tokenss that need attended to and False to padding tokenss, then add 2 dimensions
    trg_masks = (trgs != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)

    # generate subsequent masks
    trg_sub_masks = torch.tril(torch.ones((seq_lengths, seq_lengths), device = self.device )).bool()

    # bitwise "and" operator
    trg_masks = trg_masks & trg_sub_masks
    return trg_masks

  def forward(self, srcs, trgs):
    """
    Args
      trgs:        raws target sequence (batch_sizes, trg_seq_length)
      srcs:        raws srcs sequences (batch_sizes, src_seq_length)

    Returns:
      outputs:     sequences after decoders   (batch_sizes, trg_seq_length, output_dim)
    """

    # create source and target masks
    src_masks = self.make_src_masks(srcs)    #(batch_sizes, 1, 1, src_seq_length)
    trg_masks = self.make_trg_masks(trgs)    #(batch_sizes, 1, trg_seq_length, trg_seq_length)

    # push the srcs through the encoders layers
    srcs = self.encoders(self.src_embeds(srcs), src_masks)   # (batch_sizes, src_seq_length, d_model)

    # decoders outputs and attention probabilities
    outputs = self.decoders(self.trg_embeds(trgs), srcs, trg_masks, src_masks)

    return outputs

In [ ]:
def make_models(device, src_vocab, trg_vocab, n_layers: int = 3, d_model: int = 128,
               d_ffn: int = 256, n_heads: int = 8, dropout: float = 0.1,
               max_length : int = 80):
  """
    Construct a models when provided parameters.

    Args:
      src_vocab:      source vocubulary
      trg_vocab:      target vocubulary
      n_layers:       Number of encoders and decoders
      d_model:        dimension of embeddinsg
      d_ffn:          dimension of feed-forwaed network
      n_heads:        number of heads
      dropout:        probability of dropout ocurring
      max_length:     maximum sequence length for positional encodings

    Returns:
      Transformers models based on hyperparameters
  """
  encoders = Encoders(d_model, n_layers, n_heads, d_ffn, dropout)
  decoders = Decoders(len(trg_vocab), d_model, n_layers, n_heads, d_ffn, dropout)
  pos_enc_src = PositionalEncodings(d_model, dropout, max_length)
  pos_enc_trg = PositionalEncodings(d_model, dropout, max_length)

  src_embeds = nn.Sequential(Embeddingss(len(src_vocab), d_model), pos_enc_src)
  trg_embeds = nn.Sequential(Embeddingss(len(trg_vocab), d_model), pos_enc_trg)


  # create the Transformers models
  models = Transformers(encoders, decoders, src_embeds, trg_embeds,
                    src_pad_idx = src_vocab['<pad>'],
                    trg_pad_idx = trg_vocab['<pad>'],
                    device = device)

   # initialize parameters with Xavier/Glorot
  for p in models.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)

  # models.to(devices)
  return models

In [ ]:

with open("/content/drive/MyDrive/eng-tel.txt", 'r', encoding='utf-8') as file:
  raws = []
  n = 0
  for line in file:
    n +=1
    if n==5:
      break
    telugu_sentence, english_sentence = line.strip().split('>>')
    raws.append((telugu_sentence, english_sentence))

print(raws)

[('A suspicious activity alert is a notification from your bank about unusual transactions or account activity that may indicate fraud. ', ' మీ బ్యాంక్ నుండి అపరిపూర్ణ లావాదేవీలు లేదా ఖాతా చర్యలు మోసం సూచించేలా ఉన్నప్పుడు అందించే హెచ్చరిక.'), ('You received a suspicious activity alert because your bank detected unusual transactions or login attempts that differ from your normal banking behavior. ', ' మీ బ్యాంక్ సాధారణ బ్యాంకింగ్ ప్రవర్తన నుండి భిన్నంగా ఉన్న లావాదేవీలు లేదా లాగిన్ ప్రయత్నాలను కనుగొని, మీకు అనుమానాస్పద కార్యకలాప హెచ్చరిక పంపింది.'), ('If you receive a suspicious activity alert, review the transaction details, confirm whether it was authorized, and contact your bank immediately if it was unauthorized. ', ' మీరు అనుమానాస్పద కార్యకలాప హెచ్చరిక పొందితే, లావాదేవీ వివరాలను సమీక్షించండి, అది అనుమతించబడిందా లేదా నిర్ధారించుకోండి, అనుమతించబడనిదైతే వెంటనే మీ బ్యాంకును సంప్రదించండి.'), ('You can report a suspicious transaction by calling your bank’s customer service, visiting a bra

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models = make_models(device, vocab_src, vocab_trg,
                   n_layers=3, n_heads=8, d_model=256,
                   d_ffn=512, max_length=150)

models.to(device)

Transformers(
  (encoders): Encoders(
    (layers): ModuleList(
      (0-2): 3 x EncoderLayers(
        (attention): MultiHeadAttentions(
          (Wq): Linear(in_features=256, out_features=256, bias=True)
          (Wk): Linear(in_features=256, out_features=256, bias=True)
          (Wv): Linear(in_features=256, out_features=256, bias=True)
          (Wo): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (positionwise_fnn): PositionwiseFeedForwards(
          (linear_layer_1): Linear(in_features=256, out_features=512, bias=True)
          (linear_layer_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (fnn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p

In [ ]:
def count_parameters(models):
    return sum(p.numel() for p in models.parameters() if p.requires_grad)

# Now you can count the parameters
print(f'The models has {count_parameters(models):,} trainable parameters')

The models has 13,143,992 trainable parameters


In [ ]:
LEARNING_RATEs = 0.0005

optimizers = torch.optim.Adam(models.parameters(), lr = LEARNING_RATEs)
criterions = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [ ]:
def trains(models, iterator, optimizers, criterions, clip):

  # set the models to training mode
  models.train()

  epoch_losss = 0

  # loop through each batch in the iterator
  for i, batch in enumerate(iterator):

    # set the source and target batches
    srcs,trgs = batch

    # Move srcs and trgs to the same devices as the models
    srcs = srcs.to(device)
    trgs = trgs.to(device)

    # zero the gradients
    optimizers.zero_grad()

    # logitss for each outputs
    logitss = models(srcs, trgs[:,:-1])

    # expected outputs
    expected_outputs = trgs[:,1:]

    criterions.ignore_index = models.trg_pad_idx

    # logitss = logitss.to(models.devices)
    # expected_outputs = expected_outputs.to(models.devices)

    # calculate the losss
    losss = criterions(logitss.contiguous().view(-1, logitss.shape[-1]),
                    expected_outputs.contiguous().view(-1))

    # backpropagation
    losss.backward()

    # clip the weights
    torch.nn.utils.clip_grad_norm_(models.parameters(), clip)

    # update the weights
    optimizers.step()

    # update the losss
    epoch_losss += losss.item()

  # return the average losss for the epoch
  return epoch_losss / len(iterator)

In [ ]:
def evaluates(models, iterator, criterions):

  # set the models to evaluation mode
  models.eval()

  epoch_losss = 0

  # evaluates without updating gradients
  with torch.no_grad():

    # loop through each batch in the iterator
    for i, batch in enumerate(iterator):

      # set the source and target batches
      srcs, trgs = batch

      # Move srcs and trgs to the same devices as the models
      # srcs = srcs.to(models.devices)
      # trgs = trgs.to(models.devices)


      # logitss for each outputs
      logitss = models(srcs, trgs[:,:-1])

      # expected outputs
      expected_outputs = trgs[:,1:]

      # calculate the losss
      losss = criterions(logitss.contiguous().view(-1, logitss.shape[-1]),
                      expected_outputs.contiguous().view(-1))

      # update the losss
      epoch_losss += losss.item()

  # return the average losss for the epoch
  return epoch_losss / len(iterator)

In [ ]:
import torch

# Check if GPU is available
print("CUDA Available:", torch.cuda.is_available())

# Print GPU name
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Using device:", device)

CUDA Available: True
GPU: Tesla T4
Using device: cuda


In [ ]:
def epoch_times(start_times, end_times):
  elapsed_times = end_times - start_times
  elapsed_minss = int(elapsed_times / 60)
  elapsed_secss = int(elapsed_times - (elapsed_minss * 60))
  return elapsed_minss, elapsed_secss

In [ ]:
!nvidia-smi

Mon May  5 09:35:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   76C    P0             35W /   70W |     326MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
import math
import time
import torch

# Training settings
N_EPOCHSs = 10
CLIPs = 1

# ✅ Google Drive paths
best_model_path = '/content/drive/MyDrive/transformer-model_tel.pt'
checkpoint_dir = '/content/drive/MyDrive/checkpoints'
checkpoint_path = os.path.join(checkpoint_dir, 'transformer-model_tel_checkpoint.pt')

# Create checkpoints directory if it doesn't exist
os.makedirs(checkpoint_dir, exist_ok=True)

# Initialize defaults
start_epoch = 0
best_valid_losss = float('inf')

# ✅ Try to resume from checkpoints
if os.path.exists(checkpoint_path):
    checkpoints = torch.load(checkpoint_path, map_location=device)  # or use your devices

    models.load_state_dict(checkpoints['model_state_dict'])
    optimizers.load_state_dict(checkpoints['optimizer_state_dict'])
    start_epoch = checkpoints['epoch'] + 1
    best_valid_losss = checkpoints['valid_losss']
    print(f"✅ Resumed from checkpoints at epoch {start_epoch}")

# ✅ Utility to time epochs
def epoch_times(start_times, end_times):
    elapsed = end_times - start_times
    return int(elapsed // 60), int(elapsed % 60)

# ✅ Training loop
for epoch in range(start_epoch, N_EPOCHSs):
    start_times = time.time()

    train_losss = trains(models, train_iters, optimizers, criterions, CLIPs)
    valid_losss = evaluates(models, valid_iters, criterions)

    end_times = time.time()
    epoch_mins, epoch_secs = epoch_times(start_times, end_times)

    # ✅ Save best models to Drive
    if valid_losss < best_valid_losss:
        best_valid_losss = valid_losss
        torch.save(models.state_dict(), best_model_path)
        print(f"💾 Saved new best models to: {best_model_path}")

    # ✅ Save checkpoints to Drive
    torch.save({
        'epoch': epoch,
        'model_state_dict': models.state_dict(),
        'optimizer_state_dict': optimizers.state_dict(),
        'valid_losss': valid_losss,
    }, checkpoint_path)

    print(f'\n📘 Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_losss:.3f} | Train PPL: {math.exp(train_losss):7.3f}')
    print(f'\t Val. Loss: {valid_losss:.3f} |  Val. PPL: {math.exp(valid_losss):7.3f}')


💾 Saved new best models to: /content/drive/MyDrive/transformer-model_tel.pt

📘 Epoch: 01 | Time: 4m 40s
	Train Loss: 0.944 | Train PPL:   2.571
	 Val. Loss: 0.669 |  Val. PPL:   1.952
💾 Saved new best models to: /content/drive/MyDrive/transformer-model_tel.pt

📘 Epoch: 02 | Time: 4m 40s
	Train Loss: 0.622 | Train PPL:   1.862
	 Val. Loss: 0.530 |  Val. PPL:   1.699
💾 Saved new best models to: /content/drive/MyDrive/transformer-model_tel.pt

📘 Epoch: 03 | Time: 4m 39s
	Train Loss: 0.481 | Train PPL:   1.618
	 Val. Loss: 0.471 |  Val. PPL:   1.601
💾 Saved new best models to: /content/drive/MyDrive/transformer-model_tel.pt

📘 Epoch: 04 | Time: 4m 39s
	Train Loss: 0.400 | Train PPL:   1.492
	 Val. Loss: 0.429 |  Val. PPL:   1.536
💾 Saved new best models to: /content/drive/MyDrive/transformer-model_tel.pt

📘 Epoch: 05 | Time: 4m 39s
	Train Loss: 0.347 | Train PPL:   1.414
	 Val. Loss: 0.409 |  Val. PPL:   1.505
💾 Saved new best models to: /content/drive/MyDrive/transformer-model_tel.pt

📘 E

In [ ]:
# Load best models (only the state_dict, not a full checkpoints)
models.load_state_dict(torch.load('/content/drive/MyDrive/transformer-model_tel.pt', map_location=devices))
models.to(device)  # Make sure models is on the correct devices
models.eval()      # Set models to evaluation mode

# Evaluate on the test set
test_losss = evaluates(models, test_iters, criterions)

print(f'Test Loss: {test_losss:.3f} | Test PPL: {math.exp(test_losss):7.3f}')


Test Loss: 0.360 | Test PPL:   1.433


In [ ]:
# Load best models (just state_dict)
import os

file_paths = '/content/drive/MyDrive/transformer-model_tel.pt'  # Path to saved models state_dict

if os.path.exists(file_paths):
    models.load_state_dict(torch.load(file_paths, map_location=device))
    models.to(device)
    models.eval()
    print("✅ Best models loaded successfully.")
else:
    print(f"❌ Error: File not found at {file_paths}. Please check the path.")


✅ Best models loaded successfully.


In [ ]:

torch.save({
    'model_state_dict': models.state_dict(),
    'optimizer_state_dict': optimizers.state_dict(),
    'epoch': epoch,
    'valid_losss': valid_losss
}, '/content/drive/MyDrive/checkpoints/transformer-model_tel_checkpoint.pt')


In [ ]:
# Load full checkpoints (models + optimizers + metadata)
import os

checkpoint_path = '/content/drive/MyDrive/checkpoints/transformer-model_tel_checkpoint.pt'

if os.path.exists(checkpoint_path):
    checkpoints = torch.load(checkpoint_path, map_location=device)
    models.load_state_dict(checkpoints['model_state_dict'])
    optimizers.load_state_dict(checkpoints['optimizer_state_dict'])  # Optional if resuming training
    start_epoch = checkpoints['epoch'] + 1
    best_valid_losss = checkpoints['valid_losss']
    models.to(device)
    models.eval()
    print(f"✅ Checkpoint loaded. Resuming from epoch {start_epoch}")
else:
    print(f"❌ Error: Checkpoint not found at {checkpoint_path}")


✅ Checkpoint loaded. Resuming from epoch 10


In [ ]:

models.load_state_dict(torch.load('/content/drive/MyDrive/transformer-model_tel.pt'))

# calculate the losss on the test set
test_losss = evaluates(models, test_iters, criterions)

print(f'Test Loss: {test_losss:.3f} | Test PPL: {math.exp(test_losss):7.3f}')

Test Loss: 0.361 | Test PPL:   1.434


In [ ]:
models.load_state_dict(torch.load('/content/drive/MyDrive/transformer-model_tel.pt', map_location=devices))
models.to(device)
models.eval()
print("✅ Model state_dict loaded successfully from best models file.")


✅ Model state_dict loaded successfully from best models file.


In [ ]:
import os

file_paths = '/content/drive/MyDrive/transformer-model_tel.pt'  # Path to models state_dict

if os.path.exists(file_paths):
    models.load_state_dict(torch.load(file_paths, map_location=devices))
    models.to(devices)
    models.eval()
    print("✅ Loaded models state_dict successfully.")
else:
    print(f"❌ Error: File not found at {file_paths}.")


✅ Loaded models state_dict successfully.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
torch.save(models.state_dict(), '/content/drive/MyDrive/transformer2_model.pth')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

models.load_state_dict(torch.load('/content/drive/MyDrive/transformer2_model.pth'))
models.eval()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Transformers(
  (encoders): Encoders(
    (layers): ModuleList(
      (0-2): 3 x EncoderLayers(
        (attention): MultiHeadAttentions(
          (Wq): Linear(in_features=256, out_features=256, bias=True)
          (Wk): Linear(in_features=256, out_features=256, bias=True)
          (Wv): Linear(in_features=256, out_features=256, bias=True)
          (Wo): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (positionwise_fnn): PositionwiseFeedForwards(
          (linear_layer_1): Linear(in_features=256, out_features=512, bias=True)
          (linear_layer_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (fnn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p

In [ ]:
def translate_multiline(text, models, devices, vocab_src, vocab_trg, tokenizer_te, max_length=150):
    lines = text.strip().split('\n')  # Split the input text into individual lines
    all_translations = []

    for line in lines:
        if line.strip():  # Skip empty lines
            srcs, trg_tokenss = translate_sentence(line.strip(), models, devices, vocab_src, vocab_trg, tokenizer_te, max_length)
            translated_line = " ".join(trg_tokenss)
            all_translations.append(translated_line)

            # Optional: print source and target for each line
            print(f"source --> {' '.join(srcs[1:-1])}")  # remove <bos> and <eos>
            print(f"target translation --> {translated_line}\n")

    return "\n".join(all_translations)


In [ ]:

import torch

def translate_sentence(sentence, models, devices, vocab_src, vocab_trg, tokenizer_te, max_length=150):
    models.eval()

    # Check if the input is a string and tokenize accordingly

    if isinstance(sentence, str):
        # Tokenize the sentence using the Telugu tokenizer
        tokenss = tokenizer_te(sentence)
        srcs = ['<bos>'] + [token.text.lower() for token in tokenss] + ['<eos>']
    else:
        srcs = ['<bos>'] + sentence + ['<eos>']

    # Map the tokenss to their respective indices in the source vocabulary
    src_indexess = [vocab_src[token] if token in vocab_src else vocab_src['<unk>'] for token in srcs]

    # Convert the list of indices to a tensor and add a batch dimension
    src_tensors = torch.tensor(src_indexess, dtype=torch.long).unsqueeze(0).to(devices)

    # Initialize the list of target indices with the index of ''
    trg_indexess = [vocab_trg['<bos>']]

    # Initialize the loop to generate tokenss up to a maximum length
    for i in range(max_length):
        # Convert the current list of target indices to a tensor and add a batch dimension
        trg_tensors = torch.tensor(trg_indexess, dtype=torch.long).unsqueeze(0).to(devices)

        with torch.no_grad():
            # Feed the source and target tensors to the models to get the logitss
            outputs = models(src_tensors, trg_tensors)
            pred_tokens = outputs.argmax(2)[:, -1].item()

            # Check if the predicted token is '' or the maximum length is reached
            if pred_tokens == vocab_trg['<eos>'] or i == (max_length - 1):
                # Convert indices to tokenss
                trg_tokenss = [vocab_trg.lookup_token(index) for index in trg_indexess[1:]]  # Skip ''
                return srcs, trg_tokenss

            # Append the predicted token to the list of target indices
            trg_indexess.append(pred_tokens)


src_text = "how was it?"
models = models  # Replace with your actual models
devices = 'cuda' if torch.cuda.is_available() else 'cpu'  # Assuming CUDA is available and appropriate
srcs, trg_tokenss = translate_sentence(src_text, models, devices, vocab_src, vocab_trg, tokenizer_te)
print(f'source --> {" ".join(srcs[1:-1])}')
print(f'target translation --> {" ".join(trg_tokenss)}')

source --> how was it ?
target translation -->   ఎలా <unk> ?


In [ ]:
src_text = "absolutely ! i ' m sorry to hear that you ' re having trouble blocking your card . i ' m here to assist you with that . to block your card , please follow these steps 1 . contact our customer support team at {{customer support phone number}} or visit our website at {{company website url}} . 2 . provide them with the necessary details , such as your card number , expiration date , and any other required information , and any additional information , and any other relevant information  expiration date of the card"
models = models  # Replace with your actual models
devices = 'cuda' if torch.cuda.is_available() else 'cpu'  # Assuming CUDA is available and appropriate
srcs, trg_tokenss = translate_sentence(src_text, models, devices, vocab_src, vocab_trg, tokenizer_te)
print(f'source --> {" ".join(srcs[1:-1])}')
print(f'target translation --> {" ".join(trg_tokenss)}')

source --> absolutely ! i ' m sorry to hear that you ' re having trouble blocking your card . i ' m here to assist you with that . to block your card , please follow these steps 1 . contact our customer support team at { { customer support phone number } } or visit our website at { { company website url } } . 2 . provide them with the necessary details , such as your card number , expiration date , and any other required information , and any additional information , and any other relevant information   expiration date of the card
target translation -->   ఖచ్చితంగా ! మీ కార్డును తిరిగి పొందడంలో మీరు ఎదుర్కొన్న ఏ ఇతర సంబంధిత సమాచారం అయినా , మీ కార్డును తిరిగి పొందడంలో మీకు సహాయం చేయడానికి మేము చేయగలిగినదంతా చేస్తాము . మీ కార్డును నిరోధించడానికి , దయచేసి ఈ దశలను అనుసరించండి :


In [ ]:
multiline_input = """
absolutely ! i ' m sorry to hear that you ' re having trouble blocking your card .
 i ' m here to assist you with that .
 to block your card , please follow these steps
  1 . contact our customer support team at {{customer support phone number}} or visit our website at {{company website url}} .
  2 . provide them with the necessary details , such as your card number , expiration date , any other required information , any additional information , expiration date of the card.
"""

translated_text = translate_multiline(multiline_input, models, devices, vocab_src, vocab_trg, tokenizer_te)
print("Full Translation:\n", translated_text)


source --> absolutely ! i ' m sorry to hear that you ' re having trouble blocking your card .
target translation -->   ఖచ్చితంగా ! మీ కార్డును తిరిగి పొందడంలో మీరు ఎదుర్కొన్న బ్యాంక్ మేము మిమ్మల్ని <unk> ద్వారా నవీకరిస్తాము .

source --> i ' m here to assist you with that .
target translation -->   దానికి మీకు సహాయం చేయడానికి మేము ఇక్కడ ఉన్నాము .

source --> to block your card , please follow these steps
target translation -->   మీ కార్డును నిరోధించడానికి , దయచేసి ఈ దశలను అనుసరించండి :

source --> 1 . contact our customer support team at { { customer support phone number } } or visit our website at { { company website url } } .
target translation -->   1 . { { కస్టమర్ సపోర్ట్ ఫోన్ నంబర్ } at వద్ద మా కస్టమర్ మద్దతు బృందాన్ని సంప్రదించండి లేదా { { కంపెనీ వెబ్‌సైట్ url } at వద్ద మా వెబ్‌సైట్‌ను సందర్శించండి .

source --> 2 . provide them with the necessary details , such as your card number , expiration date , any other required information , any additional information , expiration date o

In [ ]:
multiline_input = """
I'm here to assist you in finding the nearest ATM in your area. To help me provide accurate information.
i ' ll be glad to assist you with that . to initiate the dispute process , please follow these steps.

"""

translated_text = translate_multiline(multiline_input, models, devices, vocab_src, vocab_trg, tokenizer_te)
print("Full Translation:\n", translated_text)


source --> i'm here to assist you in finding the nearest atm in your area . to help me provide accurate information .
target translation -->   మీ ప్రాంతంలో సమీప <unk> కనుగొనడంలో మీకు సహాయం చేయడానికి నేను ఇక్కడ ఉన్నాను . ఖచ్చితమైన సమాచారాన్ని అందించడంలో నాకు సహాయపడటానికి నేను ఇక్కడ ఉన్నాను .

source --> i ' ll be glad to assist you with that . to initiate the dispute process , please follow these steps .
target translation -->   నేను మీకు సహాయం చేయడానికి సంతోషిస్తాను . వివాద ప్రక్రియను ప్రారంభించడానికి , దయచేసి ఈ దశలను అనుసరించండి :

Full Translation:
   మీ ప్రాంతంలో సమీప <unk> కనుగొనడంలో మీకు సహాయం చేయడానికి నేను ఇక్కడ ఉన్నాను . ఖచ్చితమైన సమాచారాన్ని అందించడంలో నాకు సహాయపడటానికి నేను ఇక్కడ ఉన్నాను .
  నేను మీకు సహాయం చేయడానికి సంతోషిస్తాను . వివాద ప్రక్రియను ప్రారంభించడానికి , దయచేసి ఈ దశలను అనుసరించండి :


In [ ]:
multiline_input = """
absolutely ! i ' m here to assist you with activating your credit card for international usage .
to get started , please follow these
 steps 1 . contact our customer support team at {{customer support phone number}} or visit our website at {{company website url}} .
  2 . provide them with your credit card details , such as the card number , expiration date , and security code .
  3 . they will guide you through the necessary information , and any additional information , and any specific instructions on the necessary information and credit card details , and any additional information will be able to ensure that you can also may need to ensure that you can usually located in the necessary information can guide you should be required information required information about the necessary information about the necessary details of the necessary information

"""

translated_text = translate_multiline(multiline_input, models, devices, vocab_src, vocab_trg, tokenizer_te)
print("Full Translation:\n", translated_text)


source --> absolutely ! i ' m here to assist you with activating your credit card for international usage .
target translation -->   ఖచ్చితంగా ! అంతర్జాతీయ వినియోగం కోసం మీ క్రెడిట్ కార్డును సక్రియం చేయడంలో మీకు సహాయం చేయడానికి నేను ఇక్కడ ఉంటాను .

source --> to get started , please follow these
target translation -->   ప్రారంభించడానికి , దయచేసి ఈ దశలను అనుసరించండి :

source --> steps 1 . contact our customer support team at { { customer support phone number } } or visit our website at { { company website url } } .
target translation -->   1 . { { కస్టమర్ సపోర్ట్ ఫోన్ నంబర్ } at వద్ద మా కస్టమర్ మద్దతు బృందాన్ని సంప్రదించండి లేదా { { కంపెనీ వెబ్‌సైట్ url } at వద్ద మా వెబ్‌సైట్‌ను సందర్శించండి .

source --> 2 . provide them with your credit card details , such as the card number , expiration date , and security code .
target translation -->   2 . కార్డ్ నంబర్ , గడువు తేదీ మరియు భద్రతా కోడ్ వంటి మీ క్రెడిట్ కార్డ్ వివరాలను వారికి అందించండి .

source --> 3 . they will guide you through the

In [ ]:
multiline_input = """
Sure! I' m here to assist you with activating your card for international usage.
 Here' s what you need to do
 1. Contact our customer support team at {{customer support phone number}} or visit our website at {{company website url}} to initiate the card cancellation process.
 2. Provide the necessary details, such as your card number, card number, and other necessary information for, and any additional information.
 3. If you may be able to verify if you' ll be prepared to provide any specific card details like card details, expiration date, expiration to complete and any other required information, i' re looking for any additional information, i can vary depending on card details, card details.
 Once i' re traveling abroad, the card details, expiration date,
"""

translated_text = translate_multiline(multiline_input, models, devices, vocab_src, vocab_trg, tokenizer_te)
print("Full Translation:\n", translated_text)


source --> sure ! i ' m here to assist you with activating your card for international usage .
target translation -->   ఖచ్చితంగా ! అంతర్జాతీయ వినియోగం కోసం మీ కార్డును సక్రియం చేయడంలో మీకు సహాయం చేయడానికి నేను ఇక్కడ ఉంటాను .

source --> here ' s what you need to do
target translation -->   మీరు ఏమి చేయాలో ఇక్కడ ఉంది :

source --> 1 . contact our customer support team at { { customer support phone number } } or visit our website at { { company website url } } to initiate the card cancellation process .
target translation -->   1 . { { కస్టమర్ సపోర్ట్ ఫోన్ నంబర్ } at వద్ద మా కస్టమర్ మద్దతు బృందాన్ని సంప్రదించండి లేదా కార్డ్ రద్దు ప్రక్రియను ప్రారంభించడానికి మా వెబ్‌సైట్‌ను { { కంపెనీ వెబ్‌సైట్ url } వద్ద సందర్శించండి .

source --> 2 . provide the necessary details , such as your card number , card number , and other necessary information for , and any additional information .
target translation -->   2 . మీ కార్డ్ నంబర్ , కార్డ్ నంబర్ మరియు ఏదైనా అదనపు సమాచారం వంటి అవసరమైన వివరాలను అంది